In [ ]:
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

from models.price_evaluator_random_forest_regression import (
    PriceRegressorRandomForestHyperparams, 
    PriceRegressorRandomForestModel
)

from models.price_evaluator_xgboost_regression import (
    PriceRegressorXGBoostModelHyperparams,
    PriceRegressorXGBoostModel
)

from evaluation.evaluate_regression import evaluate_regression

from utils.load_dataset import load_dataset
from utils.load_hyperparameters import load_hyperparameters


In [ ]:
def plot_results(results):
    metrics = list(next(iter(results.values())).keys())
    model_names = list(results.keys())

    for metric in metrics:
        plt.figure()
        values = [results[model][metric] for model in model_names]
        plt.bar(model_names, values, color=['blue', 'orange'])
        plt.title(f"Comparison of {metric}")
        plt.ylabel(metric)
        plt.xlabel("Model")
        plt.show()


def main(hyperparameters_xgboost_path, hyperparameters_rf_path, dataset_path):
    X, y = load_dataset(dataset_path)
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    xgboost_hyperparams_dict = load_hyperparameters(hyperparameters_xgboost_path)
    rf_hyperparams_dict = load_hyperparameters(hyperparameters_rf_path)

    xgboost_hyperparams = PriceRegressorXGBoostModelHyperparams(**xgboost_hyperparams_dict)
    rf_hyperparams = PriceRegressorRandomForestHyperparams(**rf_hyperparams_dict)

    xgboost_model = PriceRegressorXGBoostModel(hyperparams=xgboost_hyperparams)
    rf_model = PriceRegressorRandomForestModel(hyperparams=rf_hyperparams)

    print("Training XGBoost model...")
    xgboost_model.fit(X_train, y_train)

    print("Training Random Forest model...")
    rf_model.fit(X_train, y_train)

    print("Evaluating XGBoost model...")
    xgboost_predictions = xgboost_model.predict(X_test)
    xgboost_results = evaluate_regression(xgboost_predictions, y_test)

    print("Evaluating Random Forest model...")
    rf_predictions = rf_model.predict(X_test)
    rf_results = evaluate_regression(rf_predictions, y_test)

    results = {
        "XGBoost": {
            "MAE": xgboost_results.mae,
            "MSE": xgboost_results.mse,
            "RMSE": xgboost_results.rmse,
            "R2": xgboost_results.r2,
            "MAPE": xgboost_results.mape,
        },
        "Random Forest": {
            "MAE": rf_results.mae,
            "MSE": rf_results.mse,
            "RMSE": rf_results.rmse,
            "R2": rf_results.r2,
            "MAPE": rf_results.mape,
        },
    }

    for model, metrics in results.items():
        print(f"{model} Results:")
        for metric, value in metrics.items():
            print(f"  {metric}: {value:.4f}")
        print()

    plot_results(results)

In [ ]:
hyperparameters_xgboost_path = "path/to/xgboost_hyperparameters.json"
hyperparameters_rf_path = "path/to/rf_hyperparameters.json"
dataset_path = "data/Car_sale_ads.csv"

main(hyperparameters_xgboost_path, hyperparameters_rf_path, dataset_path)